## 연습문제 Rsp

A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.

함수 | 설명
:----|:----
setA | 무작위로 가위, 바위, 보를 생성하고, 설정한다. ```1000 wei```씩 금액을 건다.
setB | 외부에서 직접 입력하여 설정한다. ```1000 wei```씩 금액을 건다.
play | 가위, 바위, 보의 승패를 정한다.
distributeBetAmount | 이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 ```1000 wei```를 가지게 된다.
getMatchResult() | "A wins", "B wins", "tie" 결과를 출력

ganache@8345에서 게임을 다음과 같이 진행하세요.

- 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력

- A가 내기금액 걸고, 컴퓨터에서 무작위로 선택한 가위바위보

- B가 내기금액 걸고, 사용자가 입력한 가위바위보

- 승부내기

- 누가 이겼는지 출력

- 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력



geth@8445에서 동일하게 실행하세요 (배점 30%)

### ganache@8345

### 1. 개발

In [9]:
%%writefile ../src/Rsp.sol
pragma solidity >=0.5.0;

contract Rsp {
    address owner;  // player A
    address addrB;  // player B
    uint8 rspA;     // 0:rock 1:scissors 2:paper
    uint8 rspB;     // 0:rock 1:scissors 2:paper
    int8 result;    // 0:tie 1:A wins 2:B wins
    uint betting;
    mapping(address=>uint) balanceOf;
    constructor() public {
        owner = msg.sender;
        betting = 0;
        result = -1;
    }
    
    function deposit(uint amount, address _addrB) payable public onlyOwner {
        require(msg.value == amount);
        balanceOf[owner] += amount;
        addrB = _addrB;
        balanceOf[addrB] += amount;
    }
    function getBalanceAll() view public returns(uint, uint, uint) {
        return (balanceOf[owner], balanceOf[addrB], address(this).balance);
    }
    function setA(uint amount) public onlyOwner {
        require(balanceOf[owner] >= amount);
        rspA = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        balanceOf[owner] -= amount;
        betting += amount;
    }
    function setB(uint8 _rsp, uint amount) public onlyOwner {
        require(balanceOf[addrB] >= amount);
        rspB = _rsp;
        balanceOf[addrB] -= amount;
        betting += amount;
    }
    function play() public {
        if (rspA == rspB)
            result = 0;     // tie
        else if ((rspA==0 && rspB==1) || (rspA==2 && rspB==2) || (rspA==2 && rspB==0))
            result = 1;     // A wins
        else
            result = 2;     // B wins
    }
    function distributeBetAmount() payable public {
        if (result == 1) {          // A wins
            balanceOf[owner] += betting;
        } else if (result == 2) {   // B wins
            balanceOf[addrB] += betting;
        } else {                    // tie
            balanceOf[owner] += betting/2;
            balanceOf[addrB] += betting/2;
        }
        betting = 0;
    }
    function getMatchResult() view public returns(string memory) {
        string memory ret = "";
        if (result == 0)
            ret = "tie";
        else if (result == 1)
            ret = "A wins";
        else if (result == 2)
            ret = "B wins";
        return ret;
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting ../src/Rsp.sol


### 2. 컴파일

In [10]:
!solc --abi --bin --gas ../src/Rsp.sol


======= ../src/Rsp.sol:Rsp =======
Gas estimation:
construction:
   47469 + 628600 = 676069
external:
   deposit(uint256,address):	65539
   distributeBetAmount():	52170
   getBalanceAll():	3780
   getMatchResult():	infinite
   play():	28418
   setA(uint256):	66616
   setB(uint8,uint256):	66438
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006002819055507fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff600160166101000a81548160ff021916908360000b60ff160217905550610c47806100a66000396000f3fe6080604052600436106100705760003560e01c806393e84cd91161004e57806393e84cd91461018c578063a0be28f5146101a3578063cb4bac76146101ad578063ee919d50146101f557610070565b80634a686381146100755780636e553f6514610105578063764e15a414610153575b600080fd5b34801561008157600080fd5b5061008a610230565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100

### 3. 배포

In [36]:
%%writefile ../src/RspDeploy.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"_addrB","type":"address"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"distributeBetAmount","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalanceAll","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getMatchResult","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"play","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setA","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint8","name":"_rsp","type":"uint8"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setB","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x"+"608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006002819055507fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff600160166101000a81548160ff021916908360000b60ff160217905550610c47806100a66000396000f3fe6080604052600436106100705760003560e01c806393e84cd91161004e57806393e84cd91461018c578063a0be28f5146101a3578063cb4bac76146101ad578063ee919d50146101f557610070565b80634a686381146100755780636e553f6514610105578063764e15a414610153575b600080fd5b34801561008157600080fd5b5061008a610230565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100ca5780820151818401526020810190506100af565b50505050905090810190601f1680156100f75780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6101516004803603604081101561011b57600080fd5b8101908080359060200190929190803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610351565b005b34801561015f57600080fd5b506101686104d8565b60405180848152602001838152602001828152602001935050505060405180910390f35b34801561019857600080fd5b506101a16105ac565b005b6101ab6106f0565b005b3480156101b957600080fd5b506101f3600480360360408110156101d057600080fd5b81019080803560ff1690602001909291908035906020019092919050505061090f565b005b34801561020157600080fd5b5061022e6004803603602081101561021857600080fd5b8101908080359060200190929190505050610a74565b005b6060806040518060200160405280600081525090506000600160169054906101000a900460000b60000b141561029d576040518060400160405280600381526020017f7469650000000000000000000000000000000000000000000000000000000000815250905061034a565b60018060169054906101000a900460000b60000b14156102f4576040518060400160405280600681526020017f412077696e7300000000000000000000000000000000000000000000000000008152509050610349565b6002600160169054906101000a900460000b60000b1415610348576040518060400160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525090505b5b5b8091505090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146103aa57600080fd5b8134146103b657600080fd5b81600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555080600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055508160036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055505050565b6000806000600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205447925092509250909192565b600160159054906101000a900460ff1660ff16600160149054906101000a900460ff1660ff1614156105fc576000600160166101000a81548160ff021916908360000b60ff1602179055506106ee565b6000600160149054906101000a900460ff1660ff1614801561062f575060018060159054906101000a900460ff1660ff16145b8061066a57506002600160149054906101000a900460ff1660ff1614801561066957506002600160159054906101000a900460ff1660ff16145b5b806106a557506002600160149054906101000a900460ff1660ff161480156106a457506000600160159054906101000a900460ff1660ff16145b5b156106cd5760018060166101000a81548160ff021916908360000b60ff1602179055506106ed565b6002600160166101000a81548160ff021916908360000b60ff1602179055505b5b565b60018060169054906101000a900460000b60000b141561077f57600254600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550610905565b6002600160169054906101000a900460000b60000b14156108105760025460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550610904565b600280548161081b57fe5b04600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550600280548161089457fe5b0460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055505b5b6000600281905550565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461096857600080fd5b8060036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205410156109d657600080fd5b81600160156101000a81548160ff021916908360ff1602179055508060036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540392505081905550806002600082825401925050819055505050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610acd57600080fd5b80600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020541015610b3a57600080fd5b6003424460405160200180838152602001828152602001925050506040516020818303038152906040528051906020012060001c81610b7557fe5b06600160146101000a81548160ff021916908360ff16021790555080600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540392505081905550806002600082825401925050819055505056fea264697066735822122008dd448f462f3b4d820366dfd8e798dfad7d4d13066a09a16652464a2908523b64736f6c63430006040033";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from "+accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data:_bin})
        .send({from:accounts[0], gas:1000000}, function(err, transactionHash) {
            if (!err)
                console.log("hash: "+transactionHash);
        });
    console.log("The contract deployed to: "+deployed.options.address)
}
deploy()

Overwriting ../src/RspDeploy.js


In [37]:
!node ../src/RspDeploy.js

Deploying the contract from 0x17632BC9dBa923a6afF6F024F09a7d1301060648
hash: 0x73163d6c01af9f61972993c261e9fe4b5b31bedc925e63801d6ee657bcfebd79
The contract deployed to: 0x61633faFBAF95018F53C0d65FfFD875A7962a07d


### 4. 사용

In [38]:
%%writefile ../src/RspUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"_addrB","type":"address"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"distributeBetAmount","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalanceAll","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getMatchResult","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"play","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setA","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint8","name":"_rsp","type":"uint8"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setB","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _rsp = new web3.eth.Contract(_abi, "0x61633faFBAF95018F53C0d65FfFD875A7962a07d");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    
    // playerB에 계정 설정 및 게임 전 playerA, B 모두 5000wei 충전
    await _rsp.methods.deposit(5000, accounts[1]).send({from:accounts[0], gas:1000000, value:5000});
    
    // 게임 전의 A, B, contract 잔고 출력
    _rsp.methods.getBalanceAll().call().then(function(v) {console.log("Before play\nA: "+v[0]+"  B: "+v[1]+"  contract: "+v[2]+"\n")});
    
    // setA(betting할 금액)
    await _rsp.methods.setA(1000).send({from:accounts[0], gas:1000000});
    
    // setB(rsp 선택, betting할 금액)
    // rsp (0:rock 1:scissors 2:paper)
    await _rsp.methods.setB(2, 1000).send({from:accounts[0], gas:1000000});
    
    // 승부내기
    await _rsp.methods.play().send({from:accounts[0], gas:1000000});
    
    // 결과 출력
    _rsp.methods.getMatchResult().call().then(console.log);
    
    // betting 금액 분배
    await _rsp.methods.distributeBetAmount().send({from:accounts[0], gas:1000000});
    
    // 게임 후의 A, B, contract 잔고 출력
    _rsp.methods.getBalanceAll().call().then(function(v) {console.log("\nAfter play\nA: "+v[0]+"  B: "+v[1]+"  contract: "+v[2])});
}
doIt()

Overwriting ../src/RspUse.js


B가 이겼을 경우

In [39]:
!node ../src/RspUse.js

Before play
A: 5000  B: 5000  contract: 5000

B wins

After play
A: 4000  B: 6000  contract: 5000


In [40]:
!node ../src/RspUse.js

Before play
A: 9000  B: 11000  contract: 10000

B wins

After play
A: 8000  B: 12000  contract: 10000


In [41]:
!node ../src/RspUse.js

Before play
A: 13000  B: 17000  contract: 15000

B wins

After play
A: 12000  B: 18000  contract: 15000


비겼을 경우

In [42]:
!node ../src/RspUse.js

Before play
A: 17000  B: 23000  contract: 20000

tie

After play
A: 17000  B: 23000  contract: 20000


In [44]:
!node ../src/RspUse.js

Before play
A: 26000  B: 34000  contract: 30000

tie

After play
A: 26000  B: 34000  contract: 30000


---
### geth@8445

geth에서 사용 시 오류 발생으로 인해
REMIX에서 `pragma solidity >=0.4.2;`로 컴파일 후 해당 ABI와 BIN을 사용했습니다.

### 배포

In [57]:
%%writefile ../src/RspDeploy-geth.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}

var _abi = [{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"},{"name":"_addrB","type":"address"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalanceAll","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_rsp","type":"uint8"},{"name":"amount","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin = "0x"+"608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006002819055507fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff600160166101000a81548160ff021916908360000b60ff160217905550610c7b806100a66000396000f3fe608060405260043610610083576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff1680634a686381146100885780636e553f6514610118578063764e15a41461016657806393e84cd91461019f578063a0be28f5146101b6578063cb4bac76146101c0578063ee919d5014610208575b600080fd5b34801561009457600080fd5b5061009d610243565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100dd5780820151818401526020810190506100c2565b50505050905090810190601f16801561010a5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6101646004803603604081101561012e57600080fd5b8101908080359060200190929190803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610365565b005b34801561017257600080fd5b5061017b6104f0565b60405180848152602001838152602001828152602001935050505060405180910390f35b3480156101ab57600080fd5b506101b46105db565b005b6101be61071f565b005b3480156101cc57600080fd5b50610206600480360360408110156101e357600080fd5b81019080803560ff16906020019092919080359060200190929190505050610942565b005b34801561021457600080fd5b506102416004803603602081101561022b57600080fd5b8101908080359060200190929190505050610aab565b005b606080602060405190810160405280600081525090506000600160169054906101000a900460000b60000b14156102b1576040805190810160405280600381526020017f7469650000000000000000000000000000000000000000000000000000000000815250905061035e565b60018060169054906101000a900460000b60000b1415610308576040805190810160405280600681526020017f412077696e730000000000000000000000000000000000000000000000000000815250905061035d565b6002600160169054906101000a900460000b60000b141561035c576040805190810160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525090505b5b5b8091505090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156103c057600080fd5b81341415156103ce57600080fd5b81600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555080600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055508160036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055505050565b6000806000600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020543073ffffffffffffffffffffffffffffffffffffffff1631925092509250909192565b600160159054906101000a900460ff1660ff16600160149054906101000a900460ff1660ff16141561062b576000600160166101000a81548160ff021916908360000b60ff16021790555061071d565b6000600160149054906101000a900460ff1660ff1614801561065e575060018060159054906101000a900460ff1660ff16145b8061069957506002600160149054906101000a900460ff1660ff1614801561069857506002600160159054906101000a900460ff1660ff16145b5b806106d457506002600160149054906101000a900460ff1660ff161480156106d357506000600160159054906101000a900460ff1660ff16145b5b156106fc5760018060166101000a81548160ff021916908360000b60ff16021790555061071c565b6002600160166101000a81548160ff021916908360000b60ff1602179055505b5b565b60018060169054906101000a900460000b60000b14156107ae57600254600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550610938565b6002600160169054906101000a900460000b60000b141561083f5760025460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550610937565b6002805481151561084c57fe5b04600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550600280548115156108c757fe5b0460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055505b5b6000600281905550565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561099d57600080fd5b8060036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205410151515610a0d57600080fd5b81600160156101000a81548160ff021916908360ff1602179055508060036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540392505081905550806002600082825401925050819055505050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16141515610b0657600080fd5b80600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205410151515610b7557600080fd5b6003424460405160200180838152602001828152602001925050506040516020818303038152906040528051906020012060019004811515610bb357fe5b06600160146101000a81548160ff021916908360ff16021790555080600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540392505081905550806002600082825401925050819055505056fea165627a7a723058205a9fc1b218a2fee24fa07f6ae132a7f0c672e8ef188a647df91480f78a5b15440029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from "+accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data:_bin})
        .send({from:accounts[0], gas:1000000}, function(err, transactionHash) {
            if (!err)
                console.log("hash: "+transactionHash);
        });
    console.log("The contract deployed to: "+deployed.options.address)
}
deploy()

Overwriting ../src/RspDeploy-geth.js


In [58]:
!node ../src/RspDeploy-geth.js

Deploying the contract from 0x74cCD5B995B3480fE9063EE539EBF065385F61C8
hash: 0xd9b51e224141812add31283d4304e6d8111d78c650f863f71b19be2416e09b28
The contract deployed to: 0x05d992E51227D7DEE49724611ddd7703d0ad1a1D


### 사용

In [59]:
%%writefile ../src/RspUse-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var fs = require('fs');

var _abi = [{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"},{"name":"_addrB","type":"address"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalanceAll","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_rsp","type":"uint8"},{"name":"amount","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _rsp = new web3.eth.Contract(_abi, "0x05d992E51227D7DEE49724611ddd7703d0ad1a1D");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    
    // playerB에 계정 설정 및 게임 전 playerA, B 모두 5000wei 충전
    await _rsp.methods.deposit(5000, accounts[1]).send({from:accounts[0], gas:3000000, value:5000});
    
    // 게임 전의 A, B, contract 잔고 출력
    _rsp.methods.getBalanceAll().call().then(function(v) {console.log("Before play\nA: "+v[0]+"  B: "+v[1]+"  contract: "+v[2]+"\n")});
    
    // setA(betting할 금액)
    await _rsp.methods.setA(1000).send({from:accounts[0], gas:3000000});
    
    // setB(rsp 선택, betting할 금액)
    // rsp (0:rock 1:scissors 2:paper)
    await _rsp.methods.setB(2, 1000).send({from:accounts[0], gas:3000000});
    
    // 승부내기
    await _rsp.methods.play().send({from:accounts[0], gas:3000000});
    
    // 결과 출력
    _rsp.methods.getMatchResult().call().then(console.log);
    
    // betting 금액 분배
    await _rsp.methods.distributeBetAmount().send({from:accounts[0], gas:3000000});
    
    // 게임 후의 A, B, contract 잔고 출력
    _rsp.methods.getBalanceAll().call().then(function(v) {console.log("\nAfter play\nA: "+v[0]+"  B: "+v[1]+"  contract: "+v[2])});
}
doIt()

Overwriting ../src/RspUse-geth.js


B가 이겼을 경우

In [60]:
!node ../src/RspUse-geth.js

Before play
A: 5000  B: 5000  contract: 5000

B wins

After play
A: 4000  B: 6000  contract: 5000


In [61]:
!node ../src/RspUse-geth.js

Before play
A: 9000  B: 11000  contract: 10000

B wins

After play
A: 8000  B: 12000  contract: 10000


In [66]:
!node ../src/RspUse-geth.js

Before play
A: 29000  B: 41000  contract: 35000

B wins

After play
A: 28000  B: 42000  contract: 35000
